# Workout: API Error Handling

## Drill 1: HTTPException 🟢
**Task:** Raise a 404 error when item not found

In [ ]:
from fastapi import HTTPException

items = {1: "Item 1", 2: "Item 2"}

# @app.get("/items/{item_id}")
def get_item(item_id: int):
    if item_id not in items:
        # Raise 404 with message "Item not found"
        raise HTTPException(status_code=404, detail="Item not found")
    return items[item_id]

---
## Drill 2: Custom Exception Class 🟡
**Task:** Create a NotFoundError exception

In [ ]:
class AppException(Exception):
    def __init__(self, message: str, code: str, status_code: int):
        self.message = message
        self.code = code
        self.status_code = status_code

class NotFoundError(AppException):
    def __init__(self, resource: str, resource_id: str | int):
        # Message: "User '123' not found"
        # Code: "NOT_FOUND"
        # Status: 404
        super().__init__(
            message=f"{resource} '{resource_id}' not found",
            code="NOT_FOUND",
            status_code=404
        )

---
## Drill 3: Exception Handler 🟡
**Task:** Register a handler for AppException

In [ ]:
from fastapi import Request
from fastapi.responses import JSONResponse

# @app.exception_handler(AppException)
async def handle_app_exception(request: Request, exc: AppException):
    return JSONResponse(
        status_code=exc.status_code,
        content={"code": exc.code, "message": exc.message}
    )

---
## Drill 4: Validation Error Handler 🔴
**Task:** Format Pydantic validation errors nicely

In [ ]:
from fastapi.exceptions import RequestValidationError

# @app.exception_handler(RequestValidationError)
async def handle_validation_error(request: Request, exc: RequestValidationError):
    # Return:
    # {"error": "VALIDATION_ERROR", "message": "...", "details": [...]}
    pass

---
## Drill 5: Global Error Handler 🟡
**Task:** Create a catch-all handler for unexpected errors

In [ ]:
# @app.exception_handler(Exception)
async def handle_unexpected(request: Request, exc: Exception):
    # Log the error
    # Return generic 500 response (don't expose internals!)
    return JSONResponse(
        status_code=500,
        content={"error": "INTERNAL_ERROR", "message": "An unexpected error occurred"}
    )

---
## Drill 6: Error Response Model 🟢
**Task:** Create a Pydantic model for error responses

In [ ]:
from pydantic import BaseModel

class ErrorResponse(BaseModel):
    error: str
    message: str
    details: list | None = None
    request_id: str | None = None

---
## Drill 7: Document Errors in OpenAPI 🟡
**Task:** Add error responses to route documentation

In [ ]:
# @app.get(
#     "/users/{user_id}",
#     responses={
#         404: {"model": ErrorResponse, "description": "User not found"},
#         401: {"model": ErrorResponse, "description": "Not authenticated"}
#     }
# )
def get_user(user_id: int):
    pass

---
## Drill 8: Exception with Headers 🟢
**Task:** Raise 401 with WWW-Authenticate header

In [ ]:
from fastapi import HTTPException

raise HTTPException(
    status_code=401,
    detail="Invalid token",
    headers={"WWW-Authenticate": "Bearer"}
)

---
## Drill 9: Request ID 🔴
**Task:** Add request ID to all error responses

In [ ]:
import uuid
from contextvars import ContextVar

request_id_var: ContextVar[str] = ContextVar("request_id")

# @app.middleware("http")
async def add_request_id(request, call_next):
    # Generate or get request ID
    # Store in context var
    # Add to response headers
    pass

---
## Drill 10: Match Exception to Status 🟢
**Task:** Match each error to its HTTP status:

In [ ]:
# A. User not authenticated    -> 401
# B. User lacks permission     -> 403
# C. Resource not found        -> 404
# D. Invalid input data        -> 400
# E. Rate limit exceeded       -> 429
# F. Validation failed         -> 422

---
## Self-Check

- [ ] Can raise HTTPException
- [ ] Can create custom exceptions
- [ ] Can register exception handlers
- [ ] Can format validation errors